# 🍽️ Restaurant Data Processing Pipeline
## Memproses Dataset `scraped_restaurants.csv` untuk Recommendation Engine

Pipeline ini bertujuan untuk:
1. Membersihkan dan memvalidasi data restaurant dari TripAdvisor
2. Melakukan feature engineering untuk rekomendasi
3. Mengekstrak entitas (lokasi, jenis makanan, menu, preferensi, features)
4. Menyiapkan data dalam format yang siap digunakan oleh recommendation engine

**Output**: `restaurants_entitas.csv` yang kompatibel dengan sistem rekomendasi yang ada.

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import ast
import re
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 2. Load and Inspect Dataset

In [2]:
# Load the scraped restaurant data
input_file = Path('scraped_restaurants.csv')
df = pd.read_csv(input_file)

print(f"📊 Dataset Info:")
print(f"   - Shape: {df.shape[0]} restaurants, {df.shape[1]} columns")
print(f"\n📋 Columns:")
print(df.columns.tolist())
print(f"\n📈 Data Types:")
print(df.dtypes)

📊 Dataset Info:
   - Shape: 1164 restaurants, 21 columns

📋 Columns:
['id', 'url', 'name', 'rating', 'about', 'address', 'address_url', 'price_range', 'cuisines', 'features', 'sunday_hours', 'monday_hours', 'tuesday_hours', 'wednesday_hours', 'thursday_hours', 'friday_hours', 'saturday_hours', 'img1_url', 'img2_url', 'img3_url', 'preferences']

📈 Data Types:
id                   int64
url                 object
name                object
rating             float64
about               object
address             object
address_url         object
price_range         object
cuisines            object
features            object
sunday_hours        object
monday_hours        object
tuesday_hours       object
wednesday_hours     object
thursday_hours      object
friday_hours        object
saturday_hours      object
img1_url            object
img2_url            object
img3_url            object
preferences         object
dtype: object


In [3]:
# Preview the data
print("📋 First 3 rows:")
df.head(3)

📋 First 3 rows:


,id,url,name,rating,about,address,address_url,price_range,cuisines,features,sunday_hours,monday_hours,tuesday_hours,wednesday_hours,thursday_hours,friday_hours,saturday_hours,img1_url,img2_url,img3_url,preferences
0,1,https://www.tripadvisor.com/Restaurant_Review-g664666-d12230759-Reviews-Gili_Teak_Resort_Restaur...,Gili Teak Resort Restaurant,4.8,We offer a stunning beach front location with amzing views over Mt Agung Bali. Come and enjoy go...,"Jalan Pantai Gili Trawangan, Pemenang, West Nusa Tenggara 83352 Indonesia","https://maps.google.com/maps?saddr=&daddr=Jalan Pantai Gili Trawangan, Pemenang, West Nusa Tengg...",$$ - $$$,"['International', 'Asian']","['Vegetarian friendly, Vegan options, Gluten free options', 'Breakfast, Lunch, Dinner, Brunch, D...",7:00 AM - 11:00 PM,7:00 AM - 11:00 PM,7:00 AM - 11:00 PM,7:00 AM - 11:00 PM,7:00 AM - 11:00 PM,7:00 AM - 11:00 PM,7:00 AM - 11:00 PM,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/29/18/a5/7e/tacos.jpg?w=900&h=500&s=1,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2e/d7/6a/7c/caption.jpg?w=900&h=500&s=1,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2d/98/16/ec/caption.jpg?w=900&h=500&s=1,"['fajitas', 'beef rendang', 'nasi goreng', 'prawns', 'coconut chicken', 'pizza', 'cobb salad', '..."
1,2,https://www.tripadvisor.com/Restaurant_Review-g664666-d27740617-Reviews-Lazy_Coconut-Gili_Trawan...,Lazy Coconut,4.9,A relaxing Beach bar and restaurant located in the north of Gili T with the most amazing sunsets...,"Jl. Pantai Gili Trawangan, Pemenang, West Nusa Tenggara 83352 Indonesia","https://maps.google.com/maps?saddr=&daddr=Jl. Pantai Gili Trawangan, Pemenang, West Nusa Tenggar...",$,['Seafood'],"['Vegetarian friendly, Vegan optionsAccepts Credit CardsLunch, Dinner', 'Vegetarian friendly, Ve...",1:00 PM - 9:30 PM,1:00 PM - 9:30 PM,1:00 PM - 9:30 PM,1:00 PM - 9:30 PM,1:00 PM - 9:30 PM,1:00 PM - 9:30 PM,1:00 PM - 9:30 PM,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/2d/bd/f3/best-sunset-in-gili-t.jpg?w=...,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2c/73/40/76/great-food.jpg?w=900&h=500&s=1,NaN,"['tacos', 'burger', 'watching the sunset', 'ate here', 'gili t', 'beach', 'beers', 'island', 'di..."
2,3,https://www.tripadvisor.com/Restaurant_Review-g664667-d12442556-Reviews-Warung_Sunny-Gili_Air_Gi...,Warung Sunny,4.6,Do you want to learn how to cook real Indonesian food. Just give me a call or text message. Only...,"Jalan yoda, Gili Indah, Pemenang, West Nusa Tenggara 83352 Indonesia","https://maps.google.com/maps?saddr=&daddr=Jalan yoda, Gili Indah, Pemenang, West Nusa Tenggara 8...",$,"['Asian', 'Healthy']","['Vegetarian friendly, Vegan options, Gluten free options', 'Lunch, Dinner', 'Delivery, Table Se...",10:00 AM - 10:00 PM,10:00 AM - 10:00 PM,10:00 AM - 10:00 PM,10:00 AM - 10:00 PM,10:00 AM - 10:00 PM,10:00 AM - 10:00 PM,10:00 AM - 10:00 PM,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/11/32/12/85/photo0jpg.jpg?w=900&h=500&s=1,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/9e/7f/5e/caption.jpg?w=900&h=500&s=1,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2a/34/0e/34/caption.jpg?w=900&h=500&s=1,"['burrito', 'rice', 'cooking class', 'indonesian food', 'amazing food', 'great local food', 'her..."


## 3. Handle Missing Values

In [4]:
# Check missing values
missing_info = df.isnull().sum()
missing_pct = (missing_info / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing_info,
    'Missing %': missing_pct
})
print("📊 Missing Values Analysis:")
print(missing_df[missing_df['Missing Count'] > 0])

📊 Missing Values Analysis:
                 Missing Count  Missing %
rating                     336      28.87
about                      183      15.72
price_range                571      49.05
sunday_hours               415      35.65
monday_hours               415      35.65
tuesday_hours              415      35.65
wednesday_hours            415      35.65
thursday_hours             415      35.65
friday_hours               415      35.65
saturday_hours             415      35.65
img1_url                   309      26.55
img2_url                   382      32.82
img3_url                   431      37.03


In [5]:
# Handle missing values
# Text fields - fill with empty string
text_columns = ['about', 'name', 'address', 'address_url', 'cuisines', 'features', 'preferences']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].fillna('')

# Rating - fill with median
if 'rating' in df.columns:
    median_rating = df['rating'].median()
    df['rating'] = df['rating'].fillna(median_rating)
    print(f"✅ Rating median: {median_rating}")

# Operating hours - fill with 'Closed'
hour_columns = [col for col in df.columns if '_hours' in col]
for col in hour_columns:
    df[col] = df[col].fillna('Closed')

# Image URLs - fill with empty string
img_columns = [col for col in df.columns if 'img' in col.lower()]
for col in img_columns:
    df[col] = df[col].fillna('')

# Price range - fill with most common
if 'price_range' in df.columns:
    df['price_range'] = df['price_range'].fillna('$$-$$$')

print(f"\n✅ Missing values handled!")
print(f"   Remaining nulls: {df.isnull().sum().sum()}")

✅ Rating median: 4.7

✅ Missing values handled!
   Remaining nulls: 0


## 4. Data Type Conversion and Cleaning

In [6]:
# Convert rating to float and ensure proper range
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0.0)
df['rating'] = df['rating'].clip(0, 5)  # Ensure rating is between 0-5

# Ensure ID is properly formatted
df['id'] = range(len(df))  # Re-index from 0

print(f"📊 Rating Statistics:")
print(f"   Min: {df['rating'].min()}")
print(f"   Max: {df['rating'].max()}")
print(f"   Mean: {df['rating'].mean():.2f}")
print(f"   Median: {df['rating'].median()}")
print(f"\n✅ Data types converted!")

📊 Rating Statistics:
   Min: 1.0
   Max: 5.0
   Mean: 4.53
   Median: 4.7

✅ Data types converted!


## 5. Parse and Clean List-like Columns

In [7]:
def safe_parse_list(value):
    """Safely parse list-like string to actual list"""
    if pd.isna(value) or value == '':
        return []
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list):
                return [str(item).strip() for item in parsed if item]
            return []
        except (ValueError, SyntaxError):
            # Try to extract items from malformed string
            items = re.findall(r"'([^']+)'", value)
            return [item.strip() for item in items if item.strip()]
    return []

# Parse list columns
list_columns = ['cuisines', 'features', 'preferences']
for col in list_columns:
    if col in df.columns:
        df[col + '_parsed'] = df[col].apply(safe_parse_list)
        print(f"✅ Parsed '{col}': {df[col + '_parsed'].apply(len).sum()} total items")

# Show sample
print("\n📋 Sample parsed cuisines:")
print(df['cuisines_parsed'].head(3).tolist())

✅ Parsed 'cuisines': 3231 total items
✅ Parsed 'features': 2708 total items
✅ Parsed 'preferences': 1526 total items

📋 Sample parsed cuisines:
[['International', 'Asian'], ['Seafood'], ['Asian', 'Healthy']]


## 6. Extract Location Entity (entitas_lokasi)

In [8]:
# Known locations in Lombok area
KNOWN_LOCATIONS = [
    'gili trawangan', 'gili air', 'gili meno', 'gili t',
    'kuta', 'kuta lombok', 'senggigi', 'mataram', 'lombok',
    'pemenang', 'mangsit', 'tanjung', 'selong belanak',
    'west nusa tenggara', 'nusa tenggara'
]

def extract_location(address):
    """Extract location from address string"""
    if pd.isna(address) or address == '':
        return 'Lombok'  # Default location
    
    address_lower = str(address).lower()
    
    # Check for specific Gili islands first
    if 'gili trawangan' in address_lower or 'gili t' in address_lower:
        return 'Gili Trawangan'
    elif 'gili air' in address_lower:
        return 'Gili Air'
    elif 'gili meno' in address_lower:
        return 'Gili Meno'
    elif 'gili indah' in address_lower:
        return 'Pemenang'
    
    # Check other locations
    if 'kuta' in address_lower:
        return 'Kuta'
    elif 'senggigi' in address_lower:
        return 'Senggigi'
    elif 'mataram' in address_lower:
        return 'Mataram'
    elif 'mangsit' in address_lower:
        return 'Senggigi'
    elif 'tanjung' in address_lower or 'medana' in address_lower:
        return 'Tanjung'
    elif 'selong belanak' in address_lower:
        return 'Selong Belanak'
    elif 'pemenang' in address_lower:
        return 'Pemenang'
    
    return 'Lombok'

# Apply location extraction
df['entitas_lokasi'] = df['address'].apply(extract_location)

# Show distribution
print("📍 Location Distribution:")
print(df['entitas_lokasi'].value_counts())

📍 Location Distribution:
entitas_lokasi
Pemenang          281
Mataram           231
Lombok            202
Kuta              147
Senggigi          129
Gili Trawangan     76
Gili Air           41
Tanjung            21
Selong Belanak     19
Gili Meno          17
Name: count, dtype: int64


## 7. Extract Food Type Entity (entitas_jenis_makanan)

In [9]:
# Food type patterns mapping
FOOD_TYPE_PATTERNS = {
    # Cuisine types
    'Italian': ['pasta', 'pizza', 'lasagna', 'risotto', 'italian', 'italia', 'tiramisu', 'gelato'],
    'Mexican': ['taco', 'burrito', 'nachos', 'quesadilla', 'mexican', 'fajita', 'enchilada'],
    'Japanese': ['sushi', 'ramen', 'tempura', 'teriyaki', 'japanese', 'japan', 'miso', 'sake'],
    'Indonesian': ['nasi goreng', 'mie goreng', 'rendang', 'sate', 'satay', 'gado-gado', 'nasi', 'indonesian'],
    'Asian': ['asian', 'stir fry', 'rice', 'noodles', 'dumpling', 'dim sum', 'asia'],
    'Thai': ['pad thai', 'tom yum', 'thai', 'curry', 'lemongrass'],
    'Mediterranean': ['mediterranean', 'hummus', 'falafel', 'kebab', 'pita'],
    'Seafood': ['seafood', 'fish', 'lobster', 'shrimp', 'prawn', 'crab', 'calamari', 'tuna', 'salmon'],
    'American': ['burger', 'steak', 'bbq', 'barbecue', 'wings', 'american', 'hot dog'],
    'Healthy': ['healthy', 'vegan', 'vegetarian', 'salad', 'smoothie', 'bowl', 'organic'],
    'Cafe': ['cafe', 'coffee', 'latte', 'espresso', 'cappuccino', 'breakfast', 'brunch'],
    'Bar': ['bar', 'cocktail', 'beer', 'wine'],
    'Pizza': ['pizza', 'pizzeria'],
    'Barbecue': ['bbq', 'barbecue', 'grill', 'grilled'],
    'French': ['french', 'france', 'croissant', 'baguette'],
    'Chinese': ['chinese', 'dim sum', 'dumpling', 'fried rice', 'wok'],
    'Indian': ['indian', 'curry', 'naan', 'tandoori', 'masala'],
    'Vietnamese': ['vietnamese', 'pho', 'banh mi', 'vietnam'],
    'European': ['european', 'europe'],
    'Fusion': ['fusion'],
}

def extract_food_types(row):
    """Extract food types from cuisines, about, and preferences"""
    text_sources = []
    
    # Add cuisines
    if 'cuisines_parsed' in row and row['cuisines_parsed']:
        text_sources.extend([c.lower() for c in row['cuisines_parsed']])
    
    # Add about text
    if 'about' in row and row['about']:
        text_sources.append(str(row['about']).lower())
    
    # Add preferences
    if 'preferences_parsed' in row and row['preferences_parsed']:
        text_sources.extend([p.lower() for p in row['preferences_parsed']])
    
    combined_text = ' '.join(text_sources)
    
    found_types = set()
    
    # First add direct cuisine matches
    if 'cuisines_parsed' in row:
        found_types.update(row['cuisines_parsed'])
    
    # Then search for pattern matches
    for food_type, patterns in FOOD_TYPE_PATTERNS.items():
        for pattern in patterns:
            if pattern.lower() in combined_text:
                found_types.add(food_type.lower())
                # Also add specific pattern if it's a food item
                if pattern not in ['asian', 'american', 'italian', 'indonesian', 'japanese', 
                                   'thai', 'chinese', 'indian', 'european', 'vietnamese', 
                                   'mexican', 'french', 'mediterranean', 'healthy', 'fusion',
                                   'cafe', 'bar']:
                    found_types.add(pattern)
                break
    
    return list(found_types)

# Apply food type extraction
df['entitas_jenis_makanan'] = df.apply(extract_food_types, axis=1)

# Show sample
print("🍜 Sample Food Types:")
for i, row in df.head(5).iterrows():
    print(f"   {row['name']}: {row['entitas_jenis_makanan'][:5]}...")

🍜 Sample Food Types:
   Gili Teak Resort Restaurant: ['cafe', 'italian', 'fajita', 'prawn', 'seafood']...
   Lazy Coconut: ['Seafood', 'burger', 'seafood', 'mexican', 'american']...
   Warung Sunny: ['mexican', 'Asian', 'burrito', 'indonesian', 'healthy']...
   El Bazar Cafe & Restaurant: ['cafe', 'seafood', 'Fast Food Restaurants in Kuta', 'Healthy Restaurants in Kuta', 'mediterranean']...
   Pearl Beach Lounge: ['Seafood', 'seafood', 'pad thai', 'pho', 'thai']...


## 8. Extract Menu Entity (entitas_menu)

In [10]:
# Menu item patterns - specific food items
MENU_PATTERNS = [
    # Main dishes
    'pizza', 'pasta', 'burger', 'steak', 'sushi', 'ramen', 'curry', 'rice',
    'noodles', 'salad', 'soup', 'sandwich', 'wrap', 'tacos', 'burrito',
    'nasi goreng', 'mie goreng', 'rendang', 'satay', 'sate', 'gado-gado',
    'pad thai', 'tom yum', 'pho', 'dumpling', 'dim sum',
    'risotto', 'lasagna', 'paella', 'falafel', 'kebab', 'shawarma',
    # Seafood
    'seafood', 'fish', 'lobster', 'shrimp', 'prawn', 'crab', 'calamari',
    'tuna', 'salmon', 'mahi mahi', 'snapper', 'octopus', 'oyster',
    # Sides & appetizers
    'fries', 'chips', 'nachos', 'spring rolls', 'tempura', 'edamame',
    'bread', 'focaccia', 'bruschetta', 'hummus',
    # Desserts
    'tiramisu', 'cheesecake', 'gelato', 'ice cream', 'cake', 'pancakes',
    # Drinks
    'coffee', 'smoothie', 'juice', 'cocktail', 'beer', 'wine',
    # Breakfast
    'breakfast', 'brunch', 'eggs', 'omelette', 'toast',
]

def extract_menu_items(row):
    """Extract specific menu items from preferences and about"""
    menu_items = set()
    
    # Check preferences first
    if 'preferences_parsed' in row and row['preferences_parsed']:
        for pref in row['preferences_parsed']:
            pref_lower = pref.lower().strip()
            for pattern in MENU_PATTERNS:
                if pattern in pref_lower or pref_lower in pattern:
                    menu_items.add(pref_lower)
                    break
    
    # Check about text
    if 'about' in row and row['about']:
        about_lower = str(row['about']).lower()
        for pattern in MENU_PATTERNS:
            if pattern in about_lower:
                menu_items.add(pattern)
    
    # Check cuisines for menu-like items
    if 'cuisines_parsed' in row:
        for cuisine in row['cuisines_parsed']:
            cuisine_lower = cuisine.lower()
            if cuisine_lower in ['pizza', 'sushi', 'barbecue', 'grill', 'seafood']:
                menu_items.add(cuisine_lower)
    
    return list(menu_items)

# Apply menu extraction
df['entitas_menu'] = df.apply(extract_menu_items, axis=1)

# Show sample
print("🍕 Sample Menu Items:")
for i, row in df.head(5).iterrows():
    print(f"   {row['name']}: {row['entitas_menu']}")

🍕 Sample Menu Items:
   Gili Teak Resort Restaurant: ['prawns', 'breakfast', 'pizza', 'cobb salad', 'nasi goreng', 'beef rendang']
   Lazy Coconut: ['tacos', 'burger', 'beers', 'seafood']
   Warung Sunny: ['burrito', 'rice']
   El Bazar Cafe & Restaurant: ['bread', 'seafood', 'breakfast', 'prawns', 'coffee']
   Pearl Beach Lounge: ['seafood', 'pad thai', 'sandwich', 'wine', 'pho', 'cocktail']


## 9. Extract Preference Entity (entitas_preferensi)

In [11]:
# Preference patterns - dining preferences/atmosphere
PREFERENCE_PATTERNS = {
    'romantic': ['romantic', 'date', 'couple', 'intimate', 'candlelit'],
    'family_friendly': ['family', 'kids', 'children', 'family-friendly', 'kid-friendly'],
    'sunset_view': ['sunset', 'sunset view', 'beach view', 'ocean view', 'sea view'],
    'rooftop': ['rooftop', 'rooftop dining', 'terrace'],
    'live_music': ['live music', 'live entertainment', 'live band'],
    'birthday': ['birthday', 'celebration', 'special occasion', 'anniversary'],
    'group_dining': ['group', 'large group', 'party', 'gathering'],
    'business': ['business', 'meeting', 'corporate'],
    'quiet': ['quiet', 'peaceful', 'tranquil', 'relaxing'],
    'lively': ['lively', 'vibrant', 'energetic', 'fun'],
    'local_experience': ['local', 'authentic', 'traditional', 'cultural'],
    'tourist_friendly': ['tourist', 'traveler', 'visitor'],
    'budget_friendly': ['budget', 'cheap', 'affordable', 'value'],
    'fine_dining': ['fine dining', 'upscale', 'luxury', 'elegant', 'premium'],
    'casual': ['casual', 'relaxed', 'informal', 'laid-back'],
    'quick_meal': ['quick', 'fast', 'takeaway', 'grab and go'],
    'healthy': ['healthy', 'organic', 'vegan', 'vegetarian', 'gluten-free'],
    'beach_dining': ['beach', 'beachfront', 'oceanside', 'seaside'],
}

def extract_preferences(row):
    """Extract dining preferences from about, features, and preferences columns"""
    prefs = set()
    
    # Combine text sources
    texts = []
    if 'about' in row and row['about']:
        texts.append(str(row['about']).lower())
    if 'preferences_parsed' in row and row['preferences_parsed']:
        texts.extend([p.lower() for p in row['preferences_parsed']])
    if 'features_parsed' in row and row['features_parsed']:
        texts.extend([f.lower() for f in row['features_parsed']])
    
    combined_text = ' '.join(texts)
    
    # Check for preference patterns
    for pref_name, keywords in PREFERENCE_PATTERNS.items():
        for keyword in keywords:
            if keyword in combined_text:
                prefs.add(pref_name)
                break
    
    return list(prefs)

# Apply preference extraction
df['entitas_preferensi'] = df.apply(extract_preferences, axis=1)

# Show sample
print("✨ Sample Preferences:")
for i, row in df.head(5).iterrows():
    print(f"   {row['name']}: {row['entitas_preferensi']}")

✨ Sample Preferences:
   Gili Teak Resort Restaurant: ['quick_meal', 'sunset_view', 'beach_dining', 'healthy']
   Lazy Coconut: ['sunset_view', 'beach_dining', 'healthy', 'quiet']
   Warung Sunny: ['healthy', 'local_experience', 'group_dining']
   El Bazar Cafe & Restaurant: ['quick_meal', 'family_friendly', 'local_experience', 'healthy']
   Pearl Beach Lounge: ['quick_meal', 'rooftop', 'healthy']


## 10. Extract Features Entity (entitas_features)

In [12]:
# Feature patterns - restaurant amenities and services
FEATURE_PATTERNS = {
    'wifi': ['wifi', 'wi-fi', 'internet', 'free wifi'],
    'outdoor_seating': ['outdoor', 'patio', 'garden', 'terrace', 'alfresco'],
    'parking': ['parking', 'valet', 'free parking'],
    'reservation': ['reservation', 'reservations', 'booking'],
    'delivery': ['delivery', 'delivers', 'takeout', 'takeaway', 'take-out'],
    'credit_card': ['credit card', 'credit cards', 'accepts credit cards'],
    'wheelchair_accessible': ['wheelchair', 'accessible', 'disability'],
    'private_dining': ['private', 'private room', 'private dining'],
    'bar': ['bar', 'full bar', 'cocktail bar'],
    'halal': ['halal', 'halal food'],
    'air_conditioned': ['air conditioning', 'air-conditioned', 'ac'],
    'seating': ['seating', 'table service', 'sit-down'],
    'pet_friendly': ['pet', 'dog friendly', 'pet friendly'],
    'smoking_area': ['smoking', 'smoking area'],
}

def extract_features(row):
    """Extract restaurant features from features column"""
    features = set()
    
    # Check features column
    if 'features_parsed' in row and row['features_parsed']:
        for feat in row['features_parsed']:
            feat_lower = feat.lower()
            for feature_name, keywords in FEATURE_PATTERNS.items():
                for keyword in keywords:
                    if keyword in feat_lower:
                        features.add(feature_name)
                        break
    
    # Check about text
    if 'about' in row and row['about']:
        about_lower = str(row['about']).lower()
        for feature_name, keywords in FEATURE_PATTERNS.items():
            for keyword in keywords:
                if keyword in about_lower:
                    features.add(feature_name)
                    break
    
    return list(features)

# Apply features extraction
df['entitas_features'] = df.apply(extract_features, axis=1)

# Show sample
print("🏪 Sample Features:")
for i, row in df.head(5).iterrows():
    print(f"   {row['name']}: {row['entitas_features']}")

🏪 Sample Features:
   Gili Teak Resort Restaurant: ['air_conditioned']
   Lazy Coconut: ['credit_card', 'air_conditioned', 'bar']
   Warung Sunny: ['delivery', 'seating', 'private_dining']
   El Bazar Cafe & Restaurant: ['delivery', 'halal', 'seating']
   Pearl Beach Lounge: ['credit_card', 'outdoor_seating', 'air_conditioned']


## 11. Data Validation & Quality Check

In [13]:
# Data validation
print("=" * 50)
print("📊 DATA VALIDATION REPORT")
print("=" * 50)

# Check for duplicates
duplicates = df[df.duplicated(subset=['name', 'address'], keep=False)]
print(f"\n🔄 Duplicate Records: {len(duplicates)}")
if len(duplicates) > 0:
    print("   Removing duplicates...")
    df = df.drop_duplicates(subset=['name', 'address'], keep='first')
    print(f"   After removal: {len(df)} records")

# Check entity coverage
print("\n📈 Entity Coverage:")
entity_cols = ['entitas_lokasi', 'entitas_jenis_makanan', 'entitas_menu', 
               'entitas_preferensi', 'entitas_features']

for col in entity_cols:
    non_empty = df[df[col].apply(lambda x: len(x) > 0 if isinstance(x, list) else bool(x))].shape[0]
    coverage = (non_empty / len(df)) * 100
    print(f"   {col}: {non_empty}/{len(df)} ({coverage:.1f}%)")

# Validate rating range
print(f"\n⭐ Rating Statistics:")
print(f"   Min: {df['rating'].min():.2f}")
print(f"   Max: {df['rating'].max():.2f}")
print(f"   Mean: {df['rating'].mean():.2f}")
print(f"   Missing: {df['rating'].isna().sum()}")

# Check required fields
print(f"\n✅ Required Fields Check:")
required_fields = ['id', 'name', 'address', 'rating']
for field in required_fields:
    missing = df[field].isna().sum()
    print(f"   {field}: {'OK' if missing == 0 else f'{missing} missing'}")

print("\n" + "=" * 50)

📊 DATA VALIDATION REPORT

🔄 Duplicate Records: 2
   Removing duplicates...
   After removal: 1163 records

📈 Entity Coverage:
   entitas_lokasi: 1163/1163 (100.0%)
   entitas_jenis_makanan: 1114/1163 (95.8%)
   entitas_menu: 436/1163 (37.5%)
   entitas_preferensi: 794/1163 (68.3%)
   entitas_features: 654/1163 (56.2%)

⭐ Rating Statistics:
   Min: 1.00
   Max: 5.00
   Mean: 4.53
   Missing: 0

✅ Required Fields Check:
   id: OK
   name: OK
   address: OK
   rating: OK



## 12. Entity Distribution Analysis

In [14]:
# Analyze entity distribution
print("📊 ENTITY DISTRIBUTION ANALYSIS")
print("=" * 50)

# Location distribution
print("\n📍 Location Distribution:")
location_counts = df['entitas_lokasi'].value_counts()
for loc, count in location_counts.head(10).items():
    print(f"   {loc}: {count}")

# Food type distribution
print("\n🍴 Food Type Distribution:")
all_food_types = []
for food_list in df['entitas_jenis_makanan']:
    if isinstance(food_list, list):
        all_food_types.extend(food_list)
food_type_counts = Counter(all_food_types)
for food_type, count in food_type_counts.most_common(10):
    print(f"   {food_type}: {count}")

# Preference distribution
print("\n✨ Preference Distribution:")
all_prefs = []
for pref_list in df['entitas_preferensi']:
    if isinstance(pref_list, list):
        all_prefs.extend(pref_list)
pref_counts = Counter(all_prefs)
for pref, count in pref_counts.most_common(10):
    print(f"   {pref}: {count}")

# Features distribution
print("\n🏪 Features Distribution:")
all_features = []
for feat_list in df['entitas_features']:
    if isinstance(feat_list, list):
        all_features.extend(feat_list)
feat_counts = Counter(all_features)
for feat, count in feat_counts.most_common(10):
    print(f"   {feat}: {count}")

📊 ENTITY DISTRIBUTION ANALYSIS

📍 Location Distribution:
   Pemenang: 280
   Mataram: 231
   Lombok: 202
   Kuta: 147
   Senggigi: 129
   Gili Trawangan: 76
   Gili Air: 41
   Tanjung: 21
   Selong Belanak: 19
   Gili Meno: 17

🍴 Food Type Distribution:
   asian: 583
   Asian: 546
   indonesian: 450
   Indonesian: 364
   healthy: 311
   italian: 273
   cafe: 244
   Arabic Restaurants in Mataram: 236
   Korean Restaurants in Mataram: 236
   bar: 216

✨ Preference Distribution:
   quick_meal: 594
   healthy: 297
   local_experience: 170
   beach_dining: 168
   sunset_view: 93
   family_friendly: 83
   casual: 32
   romantic: 27
   budget_friendly: 23
   quiet: 19

🏪 Features Distribution:
   air_conditioned: 340
   seating: 321
   delivery: 227
   credit_card: 203
   bar: 183
   outdoor_seating: 80
   private_dining: 53
   halal: 28
   reservation: 10
   wifi: 10


## 13. Prepare Final DataFrame

In [15]:
# Select and rename columns for final output
# Match format expected by recommendation engine

final_columns = [
    'id', 'name', 'rating', 'about', 'address', 'price_range',
    'cuisines', 'features', 'preferences',
    'sunday_hours', 'monday_hours', 'tuesday_hours', 'wednesday_hours',
    'thursday_hours', 'friday_hours', 'saturday_hours',
    'img1_url', 'img2_url', 'img3_url',
    'entitas_lokasi', 'entitas_jenis_makanan', 'entitas_menu',
    'entitas_preferensi', 'entitas_features'
]

# Keep original string versions of cuisines, features, preferences
# (recommendation engine uses string format)
df['cuisines'] = df['cuisines_parsed'].apply(lambda x: str(x) if x else '[]')
df['features'] = df['features_parsed'].apply(lambda x: str(x) if x else '[]')
df['preferences'] = df['preferences_parsed'].apply(lambda x: str(x) if x else '[]')

# Convert entity lists to string representation
df['entitas_jenis_makanan'] = df['entitas_jenis_makanan'].apply(lambda x: str(x) if x else '[]')
df['entitas_menu'] = df['entitas_menu'].apply(lambda x: str(x) if x else '[]')
df['entitas_preferensi'] = df['entitas_preferensi'].apply(lambda x: str(x) if x else '[]')
df['entitas_features'] = df['entitas_features'].apply(lambda x: str(x) if x else '[]')

# Select final columns
df_final = df[final_columns].copy()

print(f"📋 Final DataFrame Shape: {df_final.shape}")
print(f"\n📝 Final Columns:")
for col in final_columns:
    print(f"   - {col}")

# Preview
print("\n🔍 Sample Record:")
print(df_final.iloc[0].to_dict())

📋 Final DataFrame Shape: (1163, 24)

📝 Final Columns:
   - id
   - name
   - rating
   - about
   - address
   - price_range
   - cuisines
   - features
   - preferences
   - sunday_hours
   - monday_hours
   - tuesday_hours
   - wednesday_hours
   - thursday_hours
   - friday_hours
   - saturday_hours
   - img1_url
   - img2_url
   - img3_url
   - entitas_lokasi
   - entitas_jenis_makanan
   - entitas_menu
   - entitas_preferensi
   - entitas_features

🔍 Sample Record:
{'id': 0, 'name': 'Gili Teak Resort Restaurant', 'rating': 4.8, 'about': 'We offer a stunning beach front location with amzing views over Mt Agung Bali. Come and enjoy good food in a beautiful location. We offer Breakfast, Lunch and Dinner that can be enjoyed in our Restaurant, around the Pool or on the Beach overlooking the sea.', 'address': 'Jalan Pantai Gili Trawangan, Pemenang, West Nusa Tenggara 83352 Indonesia', 'price_range': '$$ - $$$', 'cuisines': "['International', 'Asian']", 'features': "['Vegetarian friendly

## 14. Export to CSV

In [16]:
# Export to data folder
output_path = Path('data/restaurants_entitas.csv')

# Backup existing file if exists
if output_path.exists():
    backup_path = Path('data/restaurants_entitas_backup.csv')
    import shutil
    shutil.copy(output_path, backup_path)
    print(f"📦 Backup created: {backup_path}")

# Save processed data
df_final.to_csv(output_path, index=False)
print(f"✅ Saved processed data to: {output_path}")
print(f"   Total records: {len(df_final)}")

# Also save a copy without entity columns for reference
df_base = df_final.drop(columns=['entitas_lokasi', 'entitas_jenis_makanan', 
                                  'entitas_menu', 'entitas_preferensi', 'entitas_features'])
base_output_path = Path('data/restaurants.csv')
if base_output_path.exists():
    shutil.copy(base_output_path, Path('data/restaurants_backup.csv'))
df_base.to_csv(base_output_path, index=False)
print(f"✅ Saved base data to: {base_output_path}")

📦 Backup created: data\restaurants_entitas_backup.csv
✅ Saved processed data to: data\restaurants_entitas.csv
   Total records: 1163
✅ Saved base data to: data\restaurants.csv


## 15. Final Summary

In [ ]:
# Final summary
print("=" * 60)
print("🎉 DATA PROCESSING COMPLETE!")
print("=" * 60)

print(f"""
📊 SUMMARY:
-----------
✅ Input: scraped_restaurants.csv
✅ Output: data/restaurants_entitas.csv

📈 Statistics:
   - Total restaurants: {len(df_final)}
   - Unique locations: {df_final['entitas_lokasi'].nunique()}
   - Rating range: {df_final['rating'].min():.1f} - {df_final['rating'].max():.1f}

🏷️ Entity Columns Created:
   - entitas_lokasi: Location entity (e.g., Gili Trawangan, Kuta)
   - entitas_jenis_makanan: Food type entity (e.g., Indonesia, Italian)
   - entitas_menu: Menu items (e.g., pizza, seafood)
   - entitas_preferensi: Dining preferences (e.g., romantic, family_friendly)
   - entitas_features: Restaurant features (e.g., wifi, outdoor_seating)

📁 Files Created:
   - data/restaurants_entitas.csv (processed data with entities)
   - data/restaurants.csv (base data without entities)
   - data/*_backup.csv (backups of original files)

🚀 Next Steps:
   1. Restart the recommendation engine to load new data
   2. Test chatbot responses with new restaurant data
   3. Verify entity-based filtering works correctly
""")

print("=" * 60)

🎉 DATA PROCESSING COMPLETE!

📊 SUMMARY:
-----------
✅ Input: scraped_restaurants.csv
✅ Output: data/restaurants_entitas.csv

📈 Statistics:
   - Total restaurants: 1163
   - Unique locations: 10
   - Rating range: 1.0 - 5.0

🏷️ Entity Columns Created:
   - entitas_lokasi: Location entity (e.g., Gili Trawangan, Kuta)
   - entitas_jenis_makanan: Food type entity (e.g., Indonesia, Italian)
   - entitas_menu: Menu items (e.g., pizza, seafood)
   - entitas_preferensi: Dining preferences (e.g., romantic, family_friendly)
   - entitas_features: Restaurant features (e.g., wifi, outdoor_seating)

📁 Files Created:
   - data/restaurants_entitas.csv (processed data with entities)
   - data/restaurants.csv (base data without entities)
   - data/*_backup.csv (backups of original files)

🚀 Next Steps:
   1. Restart the recommendation engine to load new data
   2. Test chatbot responses with new restaurant data
   3. Verify entity-based filtering works correctly



: 